<a href="https://colab.research.google.com/github/OttoBoop/Gr-ficos-Ciclofaixa-aos-Domingos-/blob/main/Analises_Orla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/Gabinete Flávio Valle/Pesquisa e Produção Legislativa/Faixa extra para ciclistas.csv")
df.columns

Index(['Horário do envio', 'Você costuma usar a orla para lazer aos domingos?',
       'Você é a favor da criação de uma faixa extra exclusiva de ciclistas?',
       'Você utiliza a área de lazer da orla para:',
       'Você acredita que a ciclovia atual no calçadão é suficiente para dar conta da demanda aos domingos e feriados?',
       'Se fosse criada uma ciclovia temporária na pista de rolamento, qual faixa você considera ideal?',
       'O que você gostaria de sugerir?', 'Nome', 'Telefone',
       'Endereço (com CEP)', 'Data de nascimento'],
      dtype='object')

In [6]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.4 MB/s eta 0:00:00


In [8]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [11]:
# Colab: habilitar widgets + renderer do plotly
!pip -q install ipywidgets==8.* plotly>=5
from google.colab import output
output.enable_custom_widget_manager()

import plotly.io as pio
pio.renderers.default = 'colab'  # garante que o plotly renderize no notebook


In [24]:
fig.write_html("interactive_plot.html")

In [25]:
unique_suggestions = df['O que você gostaria de sugerir?'].unique()
print(unique_suggestions)

['Praia de Botafogo ser fechada aos domingos e feriados como todas as outras '
 'Pois não passa ônibus ' nan
 'Que vcs fiscalizem! A ciclovia está uma selva! Pessoas correndo em trios, correndo em grupos, carrinhos de bebê em dupla, motos eletrônicas acima da velocidade, cada ultrapassagem ultimamente é risco de vida, adolescentes também, menores de 18 anos não deveriam poder ter bike elétrica!! Sou professora e a 15 anos dependo da minha bike para dar aula o dia todo, e digo com propriedade, as ciclovias estão um caos e muito perigosas! Precisamos urgente repensar os espaços públicos! Vcs ficam esperando algo trágico p fazer alguma cosia, não façam isso por favor, hajam logo estamos precisando muito. FISCALIZEM POR FAVOR'
 'As bicicletas são meio de transporte e não apenas utensílios de lazere. Precisam de vias exclusivas para elas. As ciclovias estao lotadas de transeuntes aleatórios, bicicletas de carga de gelo estacionadas, carrinho de milho (Todos os dias).'
 'Muitas coisas para o

In [37]:
import plotly.graph_objects as go
import plotly.io as pio
import textwrap

# --- Config & helpers ---------------------------------------------------------
questions = [
    'Você costuma usar a orla para lazer aos domingos?',
    'Você é a favor da criação de uma faixa extra exclusiva de ciclistas?',
    'Você utiliza a área de lazer da orla para:',
    'Você acredita que a ciclovia atual no calçadão é suficiente para dar conta da demanda aos domingos e feriados?',
    'Se fosse criada uma ciclovia temporária na pista de rolamento, qual faixa você considera ideal?'
]

WRAP = 18
def wrap_label(s, width=WRAP):
    return "<br>".join(textwrap.wrap(str(s), width=width))

def prep_counts(series):
    return series.fillna('Sem resposta').value_counts(dropna=False)

# --- Prepara dados agregados --------------------------------------------------
agg = {}
for q in questions:
    counts = prep_counts(df[q])
    labels = [wrap_label(x) for x in counts.index]
    agg[q] = {"labels": labels, "raw_labels": list(counts.index), "values": counts.values}

# --- Cria figura e traces (2 por pergunta: Barras + Pizza) --------------------
fig = go.Figure()
trace_map = {}
bar_idxs, pie_idxs = [], []

for qi, q in enumerate(questions):
    trace_map[q] = {}

    # Barras (visível só na pergunta inicial)
    bar_trace = go.Bar(
        x=agg[q]["labels"], y=agg[q]["values"],
        hovertemplate='%{customdata}<br><b>Respostas:</b> %{y}<extra></extra>',
        customdata=agg[q]["raw_labels"],
        name=f"{q} - Barras",
        showlegend=False,
        visible=True if (qi == 0) else False,
    )
    fig.add_trace(bar_trace)
    bar_idx = len(fig.data) - 1
    trace_map[q]["Barras"] = bar_idx
    bar_idxs.append(bar_idx)

    # Pizza (também visível na pergunta inicial; o TIPO decide qual aparece)
    pie_trace = go.Pie(
        labels=agg[q]["labels"], values=agg[q]["values"], sort=False,
        hovertemplate='%{label}<br><b>N:</b> %{value} (<b>%{percent}</b>)<extra></extra>',
        textinfo='percent+label',
        name=f"{q} - Pizza",
        showlegend=True,
        visible=True if (qi == 0) else False,
    )
    fig.add_trace(pie_trace)
    pie_idx = len(fig.data) - 1
    trace_map[q]["Pizza"] = pie_idx
    pie_idxs.append(pie_idx)

# --- Estado inicial do TIPO: Barras (opacidade e eixos) -----------------------
# Opacidade global por trace: Barras=1, Pizzas=0
for i in bar_idxs: fig.data[i].opacity = 1
for i in pie_idxs: fig.data[i].opacity = 0

# --- Botões: PERGUNTA (controlam apenas 'visible' + título) -------------------
question_buttons = []
n_traces = len(fig.data)

for q in questions:
    visible = [False] * n_traces
    visible[trace_map[q]["Barras"]] = True
    visible[trace_map[q]["Pizza"]]  = True  # mantém os dois visíveis; TIPO decide opacidade

    question_buttons.append(dict(
        label=q,
        method="update",
        args=[
            {"visible": visible},                  # só visibilidade
            {"title": q}                           # e título
        ]
    ))

# --- Botões: TIPO (não mexem em 'visible'; só opacidade, eixos e legenda) -----
# Máscaras de opacidade globais
op_barras = [1 if i in bar_idxs else 0 for i in range(n_traces)]
op_pizza  = [1 if i in pie_idxs  else 0 for i in range(n_traces)]

type_buttons = [
    dict(
        label="Barras",
        method="update",
        args=[
            {"opacity": op_barras},
            {
                "xaxis": {"visible": True,  "showgrid": True,  "gridcolor": "LightGray"},
                "yaxis": {"visible": True,  "showgrid": True,  "gridcolor": "LightGray", "rangemode": "tozero"},
                "showlegend": False
            }
        ]
    ),
    dict(
        label="Pizza",
        method="update",
        args=[
            {"opacity": op_pizza},
            {
                "xaxis": {"visible": False},
                "yaxis": {"visible": False},
                "showlegend": True
            }
        ]
    )
]

# --- Layout, menus e estilo ---------------------------------------------------
fig.update_layout(
    title=dict(text=questions[0], y=0.98, x=0.02, xanchor="left", font=dict(size=18)),
    updatemenus=[
        # Menu PERGUNTA (em cima, à esquerda)
        dict(buttons=question_buttons, direction="down", showactive=True,
             x=0.02, xanchor="left", y=1.08, yanchor="top",
             pad={"t":0,"r":0,"b":0,"l":0}),
        # Menu TIPO (logo abaixo, à esquerda)
        dict(buttons=type_buttons, direction="down", showactive=True,
             x=0.02, xanchor="left", y=1.03, yanchor="top",
             pad={"t":0,"r":0,"b":0,"l":0}),
    ],
    height=600,
    paper_bgcolor="white",
    plot_bgcolor="white",
    legend=dict(orientation="v", yanchor="top", y=1, xanchor="left", x=1.25, itemwidth=60),
    margin=dict(t=95, b=20, l=20, r=240),  # reserva espaço fixo p/ legenda
    xaxis=dict(visible=True, showgrid=True, gridcolor="LightGray"),
    yaxis=dict(visible=True, showgrid=True, gridcolor="LightGray", rangemode="tozero"),
    bargap=0.25
)

# --- (Opcional) Exportar HTML completo offline --------------------------------
pio.write_html(fig, file="grafico_interativo.html", full_html=True)

fig.show()
